### Import dependencies

In [1]:
import polars_bio as pb
import pandas as pd

/home/user/.pyenv/versions/3.12.9/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:polars_bio:Creating BioSessionContext


### Usage examples

#### Usage example - calling UDAF directly in SQL

In [2]:
a_lazyframe = pb.read_fastq("./example.fastq")
result = pb.sql("SELECT base_sequence_quality(quality_scores) FROM example").collect()
print(result.item())

INFO:polars_bio:Table: example registered for path: ./example.fastq
1rows [00:00, 327.48rows/s]

{'base_quality_warn': 'pass', 'base_per_pos_data': [{'pos': 89, 'average': 32.44, 'median': 35.0, 'q1': 33.0, 'q3': 35.0, 'lower': 30.0, 'upper': 38.0}, {'pos': 84, 'average': 32.415, 'median': 35.0, 'q1': 33.0, 'q3': 35.0, 'lower': 30.0, 'upper': 38.0}, {'pos': 96, 'average': 31.315, 'median': 34.0, 'q1': 32.0, 'q3': 35.0, 'lower': 27.5, 'upper': 39.5}, {'pos': 31, 'average': 38.795, 'median': 40.0, 'q1': 38.0, 'q3': 41.0, 'lower': 33.5, 'upper': 45.5}, {'pos': 49, 'average': 37.21, 'median': 39.0, 'q1': 36.0, 'q3': 40.0, 'lower': 30.0, 'upper': 46.0}, {'pos': 69, 'average': 33.7, 'median': 36.0, 'q1': 34.0, 'q3': 39.0, 'lower': 26.5, 'upper': 46.5}, {'pos': 63, 'average': 36.25, 'median': 38.0, 'q1': 35.0, 'q3': 40.0, 'lower': 27.5, 'upper': 47.5}, {'pos': 65, 'average': 35.995, 'median': 37.0, 'q1': 35.0, 'q3': 39.0, 'lower': 29.0, 'upper': 45.0}, {'pos': 70, 'average': 33.565, 'median': 35.0, 'q1': 34.0, 'q3': 38.0, 'lower': 28.0, 'upper': 44.0}, {'pos': 68, 'average': 35.91, 'medi

#### Usage example - .fastq file

In [3]:
pb.base_sequence_quality("example.fastq")

{'base_quality_warn': 'pass',
 'base_per_pos_data': [{'lower': 33.5,
   'q3': 41.0,
   'q1': 38.0,
   'median': 40.0,
   'average': 38.205,
   'upper': 45.5,
   'pos': 34},
  {'average': 38.47,
   'q1': 38.0,
   'lower': 33.5,
   'median': 40.0,
   'q3': 41.0,
   'pos': 18,
   'upper': 45.5},
  {'q3': 35.0,
   'average': 31.815,
   'median': 35.0,
   'q1': 33.0,
   'pos': 86,
   'lower': 30.0,
   'upper': 38.0},
  {'q3': 40.0,
   'lower': 35.0,
   'upper': 43.0,
   'pos': 21,
   'average': 38.445,
   'q1': 38.0,
   'median': 40.0},
  {'lower': 27.5,
   'average': 31.835,
   'q3': 35.0,
   'median': 35.0,
   'q1': 32.0,
   'upper': 39.5,
   'pos': 92},
  {'q3': 35.0,
   'lower': 27.5,
   'pos': 95,
   'upper': 39.5,
   'median': 34.0,
   'q1': 32.0,
   'average': 31.425},
  {'average': 35.91,
   'q3': 39.0,
   'lower': 26.5,
   'median': 36.0,
   'pos': 68,
   'upper': 46.5,
   'q1': 34.0},
  {'pos': 61,
   'average': 35.99,
   'median': 38.0,
   'q3': 40.0,
   'q1': 35.0,
   'lower': 2

#### Usage example - .csv file

In [4]:
pb.base_sequence_quality("example.csv")

{'base_quality_warn': 'pass',
 'base_per_pos_data': [{'average': 32.195,
   'pos': 85,
   'q3': 35.0,
   'lower': 30.0,
   'upper': 38.0,
   'median': 35.0,
   'q1': 33.0},
  {'average': 38.245,
   'median': 40.0,
   'q3': 40.0,
   'lower': 35.0,
   'pos': 30,
   'upper': 43.0,
   'q1': 38.0},
  {'pos': 2,
   'average': 32.015,
   'q3': 34.0,
   'lower': 26.5,
   'upper': 38.5,
   'q1': 31.0,
   'median': 34.0},
  {'lower': 30.0,
   'pos': 89,
   'upper': 38.0,
   'average': 32.44,
   'median': 35.0,
   'q1': 33.0,
   'q3': 35.0},
  {'q3': 39.0,
   'average': 37.5,
   'q1': 37.0,
   'pos': 12,
   'median': 39.0,
   'lower': 34.0,
   'upper': 42.0},
  {'median': 40.0,
   'pos': 23,
   'lower': 33.5,
   'q3': 41.0,
   'q1': 38.0,
   'upper': 45.5,
   'average': 38.635},
  {'average': 38.595,
   'median': 40.0,
   'q3': 41.0,
   'lower': 33.5,
   'q1': 38.0,
   'pos': 29,
   'upper': 45.5},
  {'average': 37.35,
   'pos': 57,
   'q1': 36.0,
   'median': 39.0,
   'lower': 30.0,
   'upper': 

#### Usage example - .parquet file

In [5]:
pb.base_sequence_quality("example.parquet")

{'base_quality_warn': 'pass',
 'base_per_pos_data': [{'average': 35.69,
   'median': 37.0,
   'q1': 35.0,
   'lower': 32.0,
   'upper': 40.0,
   'q3': 37.0,
   'pos': 3},
  {'pos': 39,
   'average': 37.895,
   'q3': 40.0,
   'lower': 35.0,
   'median': 40.0,
   'q1': 38.0,
   'upper': 43.0},
  {'pos': 20,
   'average': 38.625,
   'q3': 41.0,
   'median': 40.0,
   'lower': 33.5,
   'upper': 45.5,
   'q1': 38.0},
  {'pos': 28,
   'q1': 38.0,
   'average': 38.445,
   'lower': 33.5,
   'upper': 45.5,
   'q3': 41.0,
   'median': 40.0},
  {'pos': 49,
   'average': 37.21,
   'q1': 36.0,
   'q3': 40.0,
   'lower': 30.0,
   'median': 39.0,
   'upper': 46.0},
  {'q1': 35.0,
   'pos': 59,
   'upper': 47.5,
   'median': 38.0,
   'average': 36.08,
   'lower': 27.5,
   'q3': 40.0},
  {'pos': 67,
   'average': 35.96,
   'lower': 26.5,
   'q1': 34.0,
   'upper': 46.5,
   'q3': 39.0,
   'median': 36.5},
  {'average': 37.45,
   'median': 40.0,
   'q3': 41.0,
   'q1': 36.0,
   'lower': 28.5,
   'pos': 45

#### Usage example - `polars.lazyframe.frame.LazyFrame` object

In [6]:
a_lazyframe = pb.read_fastq("./example.fastq")
print(type(a_lazyframe))
pb.base_sequence_quality(a_lazyframe)

INFO:polars_bio:Table: example registered for path: ./example.fastq


<class 'polars.lazyframe.frame.LazyFrame'>


200rows [00:00, 126946.25rows/s]


{'base_quality_warn': 'pass',
 'base_per_pos_data': [{'average': 38.505,
   'q3': 41.0,
   'lower': 33.5,
   'median': 40.0,
   'upper': 45.5,
   'pos': 17,
   'q1': 38.0},
  {'median': 33.0,
   'average': 30.135,
   'lower': 26.5,
   'q1': 31.0,
   'q3': 34.0,
   'pos': 0,
   'upper': 38.5},
  {'lower': 32.0,
   'upper': 40.0,
   'median': 35.0,
   'pos': 5,
   'average': 35.095,
   'q1': 35.0,
   'q3': 37.0},
  {'pos': 20,
   'median': 40.0,
   'q1': 38.0,
   'q3': 41.0,
   'average': 38.625,
   'lower': 33.5,
   'upper': 45.5},
  {'pos': 10,
   'lower': 34.0,
   'q3': 39.0,
   'upper': 42.0,
   'average': 37.675,
   'q1': 37.0,
   'median': 39.0},
  {'lower': 31.0,
   'average': 37.87,
   'upper': 47.0,
   'q1': 37.0,
   'pos': 41,
   'median': 40.0,
   'q3': 41.0},
  {'pos': 56,
   'q1': 36.0,
   'lower': 30.0,
   'median': 39.0,
   'upper': 46.0,
   'q3': 40.0,
   'average': 37.59},
  {'q1': 38.0,
   'upper': 43.0,
   'average': 37.855,
   'pos': 26,
   'median': 40.0,
   'lower':

#### Usage example - `polars.dataframe.frame.DataFrame` object

In [7]:
a_lazyframe = pb.read_fastq("./example.fastq")
a_dataframe = a_lazyframe.collect()
print(type(a_dataframe))
pb.base_sequence_quality(a_dataframe)

INFO:polars_bio:Table: example registered for path: ./example.fastq
200rows [00:00, 80924.25rows/s]


<class 'polars.dataframe.frame.DataFrame'>


{'base_quality_warn': 'pass',
 'base_per_pos_data': [{'lower': 33.5,
   'upper': 45.5,
   'median': 40.0,
   'average': 38.265,
   'pos': 24,
   'q3': 41.0,
   'q1': 38.0},
  {'median': 35.0,
   'average': 30.83,
   'pos': 74,
   'q1': 32.0,
   'lower': 24.5,
   'q3': 37.0,
   'upper': 44.5},
  {'pos': 93,
   'upper': 39.5,
   'average': 31.05,
   'median': 34.0,
   'q1': 32.0,
   'q3': 35.0,
   'lower': 27.5},
  {'q1': 37.0,
   'pos': 10,
   'q3': 39.0,
   'lower': 34.0,
   'upper': 42.0,
   'median': 39.0,
   'average': 37.675},
  {'lower': 28.5,
   'upper': 48.5,
   'median': 39.0,
   'average': 37.425,
   'pos': 50,
   'q1': 36.0,
   'q3': 41.0},
  {'q1': 36.0,
   'lower': 30.0,
   'pos': 55,
   'average': 37.55,
   'q3': 40.0,
   'upper': 46.0,
   'median': 39.0},
  {'average': 32.195,
   'q1': 33.0,
   'q3': 35.0,
   'lower': 30.0,
   'pos': 85,
   'upper': 38.0,
   'median': 35.0},
  {'pos': 91,
   'q1': 32.0,
   'upper': 39.5,
   'q3': 35.0,
   'average': 32.665,
   'lower': 27

#### Usage example - `pandas.core.frame.DataFrame` object

In [8]:
a_lazyframe = pb.read_fastq("./example.fastq")
a_pandas_dataframe = a_lazyframe.collect().to_pandas()
print(type(a_pandas_dataframe))
pb.base_sequence_quality(a_pandas_dataframe)

INFO:polars_bio:Table: example registered for path: ./example.fastq
200rows [00:00, 120508.66rows/s]

<class 'pandas.core.frame.DataFrame'>


{'base_quality_warn': 'pass',
 'base_per_pos_data': [{'upper': 41.0,
   'q3': 35.0,
   'lower': 25.0,
   'pos': 97,
   'average': 30.67,
   'median': 34.0,
   'q1': 31.0},
  {'q1': 38.0,
   'pos': 14,
   'average': 38.965,
   'lower': 33.5,
   'median': 40.0,
   'upper': 45.5,
   'q3': 41.0},
  {'pos': 23,
   'upper': 45.5,
   'q1': 38.0,
   'q3': 41.0,
   'average': 38.635,
   'lower': 33.5,
   'median': 40.0},
  {'q1': 37.0,
   'lower': 34.0,
   'average': 37.71,
   'upper': 42.0,
   'q3': 39.0,
   'median': 39.0,
   'pos': 11},
  {'average': 38.425,
   'q1': 38.0,
   'q3': 41.0,
   'lower': 33.5,
   'upper': 45.5,
   'median': 40.0,
   'pos': 19},
  {'q3': 34.0,
   'upper': 38.5,
   'average': 32.015,
   'q1': 31.0,
   'lower': 26.5,
   'pos': 2,
   'median': 34.0},
  {'upper': 48.5,
   'q1': 36.0,
   'average': 37.45,
   'median': 40.0,
   'pos': 45,
   'q3': 41.0,
   'lower': 28.5},
  {'average': 37.61,
   'pos': 48,
   'median': 40.0,
   'q1': 36.0,
   'upper': 48.5,
   'q3': 41.